In [ ]:
# Load a ROMS grid
import os, sys
from gridtools.gridutils import GridUtils
from gridtools.grids import roms
from pyproj import Proj
import cartopy.crs as crs
import xarray as xr

# Set a place to write files
os.environ["PYROMS_GRIDID_FILE"] = "/import/AKWATERS/jrcermakiii/configs/Arctic6/roms/gridid.txt"
wrkDir = '/import/AKWATERS/jrcermakiii/configs/zOutput'
inputDir = os.path.join(wrkDir, 'INPUT')

grd = GridUtils()

romsObj = roms.ROMS()
romsGrd = romsObj.get_ROMS_grid('ARCTIC6')

In [ ]:
ds = xr.Dataset()
ds['lat'] = (('ny', 'nx'), romsGrd.hgrid.lat_rho)
ds['lon'] = (('ny', 'nx'), romsGrd.hgrid.lon_rho)
ds['mask'] = (('ny', 'nx'), romsGrd.hgrid.mask)

# Enable 2D mapping of coordinates
ds = ds.set_coords(['lat', 'lon'])

In [ ]:
# Informal mask editor 

# This does not change anything yet

import xarray as xr #version 0.15.1
import hvplot.xarray #version 0.6.0
import holoviews as hv
import geoviews as gv #version 1.8.1
import cartopy.crs as ccrs
import numpy as np
gv.extension('bokeh')

clickedValues = []
lastClickY = 204
lastClickX = 820
gridRange = 50

def getGridSubset(lastY, lastX, grd):
    grdShape = grd.shape
    
    firstY = lastY - gridRange
    lastY = lastY + gridRange
    firstX = lastX - gridRange
    lastX = lastX + gridRange
    
    if firstY < 0:
        firstY = 0
    if firstX < 0:
        firstX = 0
    
    if lastY > grdShape[0]-1:
        lastY = grdShape[0]-1
    
    if lastX > grdShape[1]-1:
        lastX = grdShape[1]-1
    
    return [firstY, lastY, firstX, lastX]

    
# convert to numpy
def great_circle(lon1, lat1, lon2, lat2):
    # REF: https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97
    # Convert to numpy
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])


    dist = 6371 * (
        np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon1 - lon2))
    )
    
    return dist

def plot(x, y):
    
    global da, lastClickX, lastClickY
    
    lon = x
    lat = y

    if x is not None:
        
        # Curvilinear
        d = great_circle(lon, lat, da['lon'], da['lat'])
        ind = np.nonzero(np.equal(d, np.amin(d)))
        yloc = int(ind[0][0])
        xloc = int(ind[1][0])
        
        clickedValues.append([lon, lat, yloc, xloc])
        # Make sure the dimensions match
        # da_sel = da.sel(ny = yloc, nx = xloc)        
        da.loc[yloc, xloc] = np.where(da.loc[yloc, xloc] == 0, 1, 0)
        
        lastClickY = yloc
        lastClickX = xloc
        
    # Use subset method to edit grids
    [gy1, gy2, gx1, gx2] = getGridSubset(lastClickY, lastClickX, da)

    # This has a side effect of dynamically changing the colorbar and the
    # rendered image.
    
    # REF: https://hvplot.holoviz.org/user_guide/Geographic_Data.html#declaring-an-output-projection
    plt = da.sel(ny = slice(gy1, gy2), nx = slice(gx1, gx2)).hvplot.quadmesh(
        'lon', 'lat', projection=ccrs.Orthographic(-90, 90),
        global_extent=True, frame_height=540, cmap='viridis',
        coastline='10m'
    )
    
    return plt

da = ds["mask"]

tap_stream = hv.streams.Tap()
dmap = gv.DynamicMap(plot, streams=[tap_stream])
dmap